In [110]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from itertools import permutations, product
import functools
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score, precision_recall_curve, auc
import tqdm
from numba import njit,jit
import numpy as np
from numpy import mean
from numpy import std
import pickle
from os import path
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
from collections import OrderedDict
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)



In [68]:
df = pd.read_csv('virus_genome/human_infecting_virus', delimiter='\t', header=None)
df[['ID', 'DNA Sequence']] = df[0].str.split(expand=True)
df = df.drop(0, axis=1)
df['isZoonotic'] = 1


df2 = pd.read_csv('virus_genome/Other_viruses', delimiter='\t', header=None)
df2[['ID', 'DNA Sequence']] = df2[0].str.split(expand=True)
df2 = df2.drop(0, axis=1)
df2['isZoonotic'] = 0

# make both have the same amount of samples..?

thresh = len(df)
# df2.drop(df2.index[thresh:len(df2.index)], inplace=True)

dfother = pd.DataFrame()
it = len(df)+1

print(len(df))
print(len(df2))
# iterate through each sequence
for i in range(len(df2)):
    dfother.at[it, 'DNA Sequence']=df2.at[i, 'DNA Sequence']
    dfother.at[it, 'isZoonotic']=int(0)
    dfother.at[it, 'ID']=df2.at[i, 'ID']

    it+=1


mergedDf = pd.concat([df, dfother], axis=0)


1236
8192


In [69]:
print(mergedDf)

           ID                                       DNA Sequence  isZoonotic
0       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
1      129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...         1.0
2       59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...         1.0
3      120686  ttgttctacttcttactcattattataaattataatgtttgtataa...         1.0
4       99565  gtgaatgatgatggcgtcgaaagacgtcgttgcagctactgctgct...         1.0
...       ...                                                ...         ...
9424   268315  ggatccacagaactccttgtatgtacagccgcgggtacccacagca...         0.0
9425   130329  ctcatgtcattaataagaccatgcagaaaatgcaaatgaggcgaag...         0.0
9426  1239437  ccgaagtgggatggcggctgccgccgcttcggcgcttggcgcttcg...         0.0
9427   358812  gtatacgaggttagttcattctcgtatacacgattggacaaatcaa...         0.0
9428    11287  atggaaggaggaattcgtgcagcgttttcaggcctgaatgatgtta...         0.0

[9428 rows x 3 columns]


In [70]:
def resetkmerdict(permset)->OrderedDict:
        kmerdict = OrderedDict()
        for i in permset:
            kmerdict[i]=0
        return kmerdict

def assign_kmers_to_dict(row, permset, kmer):
    kmerdict=resetkmerdict(permset)
    st = row[2] # tune for which column the sequence is in
    for j in range(len(st)-kmer+1):
        if not st[j:j+kmer] in permset: continue
        kmerdict[st[j:j+kmer]]+=1
    return kmerdict

def getTrainParams(mergedDf):
    kmer = 4
    print(mergedDf)
    s = product('acgt',repeat = kmer)
    permset = set(["".join(x) for x in list(s)])
    # print(permset)

    l = []
    
    for row in tqdm.tqdm(mergedDf.itertuples()):
        # print(row)
        l.append(assign_kmers_to_dict(row, permset, kmer))
        

    finalkmerdict=pd.DataFrame(l)
    # print(finalkmerdict)

    # print("finished")
    mergedDf.fillna(0, inplace=True)

    X = finalkmerdict
    Y = mergedDf['isZoonotic']

    print(X)
    print(Y)
    return train_test_split(X, Y, test_size=0.2, random_state=1)

In [71]:
X_train, X_test, y_train, y_test = getTrainParams(mergedDf)

zz = X_train.head()

{'tcgg', 'gggg', 'caac', 'accc', 'acgc', 'atga', 'aaat', 'cgat', 'aggt', 'ttca', 'tctt', 'ctag', 'ataa', 'tact', 'gaaa', 'caaa', 'cgtt', 'accg', 'ttaa', 'gcca', 'ctcg', 'gatg', 'gcgc', 'caga', 'taat', 'gctt', 'atcg', 'agtc', 'tagt', 'tgcc', 'ctac', 'gagg', 'acta', 'actc', 'cgac', 'ttac', 'gaac', 'aatt', 'gttg', 'tcgc', 'gtgg', 'ccac', 'tagg', 'ggtg', 'gcaa', 'agga', 'gtgt', 'ccgc', 'ttga', 'cacg', 'ttat', 'aaag', 'cgcc', 'tgac', 'tagc', 'cggg', 'taaa', 'ttcc', 'tctg', 'actg', 'atgc', 'gtat', 'ccgg', 'gaag', 'catg', 'tcac', 'tatt', 'tcct', 'aggc', 'ctca', 'tgat', 'gatc', 'acct', 'agcg', 'ctcc', 'cgca', 'ttct', 'tggc', 'ggaa', 'gtac', 'ctgt', 'cagg', 'ggct', 'ggac', 'gcag', 'gtcg', 'tcgt', 'attg', 'tcta', 'tttt', 'acca', 'acgt', 'cagt', 'tgta', 'cctg', 'tacg', 'acga', 'cata', 'gcct', 'tggg', 'acgg', 'agta', 'agct', 'atgg', 'ggcg', 'gtta', 'cacc', 'caca', 'gctg', 'tttg', 'gata', 'ggta', 'gccc', 'caag', 'attt', 'atct', 'tgca', 'ggag', 'ggat', 'tacc', 'gcta', 'gacc', 'aatg', 'gatt', 'gtcc',

9428it [01:14, 126.04it/s]


      tcgg  gggg  caac  accc  acgc  atga  aaat  cgat  aggt  ttca  ...  gccg  \
0        2    19    39    21    12    30    63     9    34    22  ...     5   
1        4    26    34    27    12    33    68     8    40    21  ...     7   
2       33    21    72    51    49    61    43    27    47    35  ...    58   
3        7    29    30    33    16    31    48    10    38    21  ...    13   
4        8    33    41    47    10    50    39     6    27    27  ...    20   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
9423     5     8    14     9     2    13    14     7    10    12  ...    10   
9424    90    81   113    62    66   157   289    70   118   181  ...   100   
9425    10    21    42    15    11    47    55     4    37    33  ...    12   
9426    15    73    74    58    15    85    70    23    61    42  ...    27   
9427    29    87    74    53    15    62    39    23    40    54  ...     8   

      cctc  gtgc  ggca  gcgg  ctct  tgag  tgga  gac

In [72]:
model = LogisticRegression(
    multi_class="multinomial", max_iter=1000,
    fit_intercept=False, tol=0.001, solver='saga', random_state=42)
model.fit(X_train, y_train)

LogisticRegression(fit_intercept=False, max_iter=1000,
                   multi_class='multinomial', random_state=42, solver='saga',
                   tol=0.001)

In [85]:
# all 0s for some reason
print(X_test[:10])
predictions = model.predict(X_test)
print(accuracy_score(y_test, predictions))

      tcgg  gggg  caac  accc  acgc  atga  aaat  cgat  aggt  ttca  ...  gccg  \
1302     8     7     6     3     4    19    24     5    12     9  ...     4   
6596    23     8    25    14     9    59   100    49    31    67  ...    12   
4095     5     5     4     6     6    22    27     8    15    10  ...     5   
7284   576   147   839   375   345   917  1590   809   340  1507  ...   401   
4899     5     2    43    37    15   111    70    38    22    51  ...    13   
6289    25    26    40    26    16   110   100    28    30    57  ...     7   
7830   431   163   280   273   330   120    22   349   160    97  ...   803   
2048   466   221   245   301   301   129    19   354   198   155  ...   503   
7122     3    28   141    64   106   320   445    53   151   354  ...    17   
7488     6    12     7    12     2    12     8     2     9    18  ...    12   

      cctc  gtgc  ggca  gcgg  ctct  tgag  tgga  gacg  ctga  
1302     8     9     6     6    13    11    11     4    13  
6596    

In [74]:
knntest = BalancedBaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors = 1, n_jobs = 1), n_estimators = 10, n_jobs = 1)

knntest.fit(X_train, y_train)
# knn = KNeighborsClassifier(n_neighbors=5)

BalancedBaggingClassifier(base_estimator=KNeighborsClassifier(n_jobs=1,
                                                              n_neighbors=1),
                          n_jobs=1)

In [75]:
predictions = knntest.predict(X_test)
print(predictions[:10])
print(accuracy_score(y_test, predictions))

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
0.9220572640509014


In [76]:

randforest = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(max_features="auto"))

randforest.fit(X_train, y_train)

/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(max_features='auto'))

In [77]:
def saveModel(model, name, X_test, y_test, params=None):
    predictions = model.predict(X_test)
    currAcc = accuracy_score(y_test, predictions)
    
    print(confusion_matrix(y_test, predictions))
    
    if not path.exists(f"{name}.pkl"):
        print("does not exist")
        pickle.dump(model, open(f'{name}.pkl', 'wb'))
    else:
        pickled_model = pickle.load(open(f'{name}.pkl', 'rb'))
        picklePredictions=pickled_model.predict(X_test)
        pickleAcc=accuracy_score(y_test, picklePredictions)
        
        if currAcc > pickleAcc:
            print("update!")
            print(confusion_matrix(y_test, picklePredictions))

            print("curr", currAcc, "pickle", pickleAcc)
            pickle.dump(model, open(f'{name}.pkl', 'wb'))

            if params != None:
                pickle.dump(params, open(f'{name}-params.pkl', 'wb'))
        else:
            model=pickled_model
    return model

In [78]:
saveModel(randforest, "randforest", X_test, y_test)

predictions = randforest.predict(X_test)
print(accuracy_score(y_test, predictions))

# for i in range(1000):
#     predictions[i] != y_test_list[i] and print(predictions[i], "vs", y_test_list[i])

[[1559   75]
 [  38  214]]
update!
[[1535   99]
 [  45  207]]
curr 0.940084835630965 pickle 0.9236479321314952
0.940084835630965


In [79]:
probability_predictions = randforest.predict_proba(X_test)
preds = randforest.predict(X_test)

print(probability_predictions[:10])
print(preds[:10])
print(y_test[:10])


[[1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [0.9 0.1]
 [0.7 0.3]
 [1.  0. ]
 [1.  0. ]
 [1.  0. ]
 [0.5 0.5]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1303    0.0
6597    0.0
4096    0.0
7285    0.0
4900    0.0
6290    0.0
7831    0.0
2049    0.0
7123    0.0
7489    0.0
Name: isZoonotic, dtype: float64


/var/folders/vf/6r913rfx607_3y64x5l2zw4r0000gn/T/ipykernel_11984/132363789.py:6: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  print(y_test[:10])


In [100]:
lr_list = [0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65,0.7, 0.75, 0.8, 0.85, 1]

parameters={
   "n_estimators":100, # 200 kind of overfits I think
    "max_features":2,
    "max_depth":6,
    "random_state":0,
    "min_sample_split":50,
    "subsample":0.8,
    "warm_start":True
}



for learning_rate in lr_list:
    gradBoost = GradientBoostingClassifier(n_estimators=parameters['n_estimators'], 
    learning_rate=learning_rate, max_features=parameters['max_features'], 
    max_depth=parameters['max_depth'], random_state=parameters['random_state'], 
    min_samples_split=parameters['min_sample_split'], subsample=parameters['subsample'], 
    warm_start=parameters['warm_start'])

    parameters['learning_rate']=learning_rate
    gradBoost.fit(X_train, y_train)

    testingAcc = accuracy_score(y_test, gradBoost.predict(X_test))
    trainingAcc = accuracy_score(y_train, gradBoost.predict(X_train))
    
    print("Learning rate: ", learning_rate)
    # print("Accuracy score (training): {0:.3f}".format(trainingAcc))
    print("Accuracy score (validation): {0:.3f}".format(testingAcc))
    # print(f"Feature importance {gradBoost.feature_importances_}")

    # pickle.dump(gradBoost, open('gradBoost.pkl', 'wb'))
    saveModel(gradBoost, "gradBoost", X_test, y_test, parameters)

Learning rate:  0.05
Accuracy score (validation): 0.957
[[1626    8]
 [  74  178]]
Learning rate:  0.075
Accuracy score (validation): 0.955
[[1621   13]
 [  71  181]]
Learning rate:  0.1
Accuracy score (validation): 0.955
[[1618   16]
 [  68  184]]
Learning rate:  0.15
Accuracy score (validation): 0.959
[[1618   16]
 [  62  190]]
Learning rate:  0.2
Accuracy score (validation): 0.955
[[1610   24]
 [  60  192]]
Learning rate:  0.25
Accuracy score (validation): 0.955
[[1609   25]
 [  60  192]]
Learning rate:  0.3
Accuracy score (validation): 0.951
[[1614   20]
 [  72  180]]
Learning rate:  0.35
Accuracy score (validation): 0.953
[[1612   22]
 [  67  185]]
Learning rate:  0.4
Accuracy score (validation): 0.956
[[1610   24]
 [  59  193]]
Learning rate:  0.45
Accuracy score (validation): 0.959
[[1614   20]
 [  57  195]]
Learning rate:  0.5
Accuracy score (validation): 0.951
[[1598   36]
 [  57  195]]
Learning rate:  0.55
Accuracy score (validation): 0.955
[[1606   28]
 [  56  196]]
Learning

In [101]:
# Models within "orig_models" folder have the highest training acc but not as great cross val

best_grad_boost = pickle.load(open('gradBoost.pkl', 'rb'))
grad_params = pickle.load(open('gradBoost-params.pkl', 'rb'))

print(grad_params)

best_rf = pickle.load(open('randforest.pkl', 'rb'))

print("Best Gradient Boosting Accuracy: ", accuracy_score(y_test, best_grad_boost.predict(X_test)))
print("Best Random Forest Accuracy: ", accuracy_score(y_test, best_rf.predict(X_test)))

print("Balanced Gradient Boosting Accuracy: ", balanced_accuracy_score(y_test, best_grad_boost.predict(X_test)))
print("Balanced Random Forest Accuracy: ", balanced_accuracy_score(y_test, best_rf.predict(X_test)))

{'n_estimators': 100, 'max_features': 2, 'max_depth': 8, 'random_state': 0, 'min_sample_split': 50, 'subsample': 0.8, 'warm_start': True, 'learning_rate': 0.4}
Best Gradient Boosting Accuracy:  0.9618239660657476
Best Random Forest Accuracy:  0.940084835630965
Balanced Gradient Boosting Accuracy:  0.8823147986244682
Balanced Random Forest Accuracy:  0.9016533582017058


In [112]:
y_thing = y_test.to_numpy()
precision, recall, thresholds = precision_recall_curve(y_thing, best_grad_boost.predict(X_test))
area = auc(recall, precision)
print(area)

0.8663018229561179


In [99]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(best_grad_boost, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


Accuracy: 0.951 (0.014)


In [84]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(best_rf, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/Users/benjaminli/opt/miniconda3/envs/seq/lib/python3.8/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be

Accuracy: 0.911 (0.017)
